In [2]:
import pickle
with open(file='gpt_data.pkl', mode='rb') as f:
    gpt_data=pickle.load(f)

In [13]:
news_less2000 = [news for news in gpt_data if len(news[0]) < 2000]

In [14]:
instruction_lists = []
output=[]
for news in news_less2000:
    instruction_lists.append(news[0])
    output.append(str(news[1]))

In [21]:
import pandas as pd
import json
df = pd.DataFrame()
df['instruction'] = instruction_lists
df['input'] = ''
df['output'] = output

In [7]:
import re
import ast

In [22]:
# DataFrame에서 'instruction'과 'output' 열을 딕셔너리 리스트로 변환
data_to_save = df[['instruction', 'input', 'output']].to_dict(orient='records')

# JSON 파일로 저장
with open('company_extract.json', 'w', encoding='utf-8') as f:
    json.dump(data_to_save, f, ensure_ascii=False, indent=4)

In [11]:
import json
with open('company_extract.json', 'r', encoding='utf-8') as f:
    ss = json.load(f)

In [13]:
%pwd

'/Users/irene/Projects/LLaMA-Factory/data'

In [8]:
new_jsons = []
for s in ss:
    new_json= {}
    inst = s['instruction'] +'\n 회사리스트 :'
    inpu = s['input']
    output = s['output']
    output = re.sub('회사리스트 ?: ?','',output)

    output_list = ast.literal_eval(output)
    output_str = str(output_list)
    new_json['instruction'] = inst
    new_json['input'] = ''
    new_json['output'] = output_str
    new_jsons.append(new_json)

    


In [10]:
# JSON 파일로 저장
with open('company_extract.json', 'w', encoding='utf-8') as f:
    json.dump(new_jsons, f, ensure_ascii=False, indent=4)

In [ ]:
## src>llama_factory>data>template.py 추가


# copied from llama3 template

_register_template(
    name="llama3-extract-company",
    format_user=StringFormatter(
        slots=[
            (
                "<|start_header_id|>user<|end_header_id|>\n\n{{content}}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>"
            )
        ]
    ),
    format_system=StringFormatter(
        slots=[{"bos_token"}, "<|start_header_id|>system<|end_header_id|>\n\n{{content}}<|eot_id|>"]
    ),
    format_observation=StringFormatter(
        slots=[
            (
                "<|start_header_id|>tool<|end_header_id|>\n\n{{content}}<|eot_id|>"
                "<|start_header_id|>assistant<|end_header_id|>\n\n"
            )
        ]
    ),
    default_system="당신은 주어진 뉴스 기사에서 언급된 회사명을 나열하는 모델입니다.\n\n1. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.\n\n2. 회사명을 찾지 못한경우 파이썬 empty list 를 출력하세요. \n\n## 뉴스 기사",
    stop_words=["<|eot_id|>"],
    replace_eos=True,
)    